In [7]:
import pickle
import os
from os import listdir
from os.path import isfile, join
import numpy as np
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
import time
import pandas as pd
from sklearn import preprocessing
from IPython.display import Image
from IPython.core.display import HTML 

#!pip install memory_profiler
#%load_ext memory_profiler

No início do projecto contava com um dataset com 1099 ficheiros audio de fado extraídos do youtube em formato .wav bem como um conjunto de 3 tipos de features já produzidas pelo Dorin para cada uma das músicas. Para explorar, visualizei os dados com tsne 2D e 3D. Também limpei o ficheiro das labels para incluir apenas ID - Nome de Artista - Nome de canção. Havia também ficheiros muito grandes (álbuns inteiros) que removi do dataset.




![tsne_chroma_2D](Resources/phase0/Chroma/tsne/tsne_chroma.jpg)

<b>Imagem 1:</b> TSNE às médias e variâncias das features Chroma de cada faixa de música. Os pontos estão coloridos por artista. O tsne não revela nenhum tipo de estrutura para além de dois grandes clusters.

A ideia inicial era aplicar uma implementação de gaussian LDA - no mirtools - a estas features contínuas para explorar possíveis tópicos que se apresentassem. Nesta fase inicial estava a escrever scripts de Bash e pequenos programas em C++ para tratar os dados, fazer visualizações, e transformações necessárias, que - em retrospectiva - consumiu muito tempo. 


![glda_chroma_40_topics](Resources/phase0/Chroma/gLDA/glda_chroma_40_topics_40_iter.png)

<b>Imagem 2:</b>  Coeficientes dos tópicos em cada faixa

y: tópico - x: faixa

Como é evidente, os resultados foram anómalos e foi passado bastante tempo a rever scripts, aprender mais sobre e voltar a gerar as features em MFCC e Chroma. 


In [8]:
def invlogamplitude(S):
    """librosa.logamplitude is actually 10_log10, so invert that."""
    return 10.0**(S/10.0)

# retorna nome de wavs em directório
def getWavNames(inputDir):
    wavFileNames = [join(inputDir, f) for f in listdir(inputDir) if isfile(join(inputDir, f)) and f[-3:]=="wav"]
    #usableIDs = [int(file[-8:-4]) for file in wavFileNames]

    return wavFileNames

#n_mfcc: número de coeficientes desejados
#nfft: comprimento para a fft, normalmente é a potência de 2 seguinte ao comprimento do maior sinal
nfft = 256
def makeMFCC(inputDir, outputDir, n_mfcc=20):
    wavNames = getWavNames(inputDir)
    
    for filename in wavNames:
        # load
        y, sr = librosa.load(filename)

        # calculate mfcc
        mfccs = librosa.feature.mfcc(y, sr=sr, n_mfcc=n_mfcc)
        #mfccName = filename[-8:-4] + "_mfcc.csv"
        mfccName = os.path.basename(filename)[:-4] + "_mfcc.csv"
        saveDir = outputDir + mfccName
        mfccs = pd.DataFrame(mfccs.T)
        d.to_csv(saveDir)
        print(filename + " " + str(mfccs.shape))
        
def makeChroma(inputDir, outputDir, nfft = 256):
    wavNames = getWavNames(inputDir)
    
    for filename in wavNames:
        #load
        y, sr = librosa.load(filename)
        
        #calculate chroma features
        chroma = librosa.feature.chroma_stft(y=y, sr=sr, n_fft = nfft, hop_length = int(nfft/2))
        chromaName = os.path.basename(filename)[:-4] + "_chroma.csv"
        saveDir = outputDir + chromaName
        chroma = pd.DataFrame(chroma.T)
        chroma.to_csv(saveDir)
        print(filename + " " + str(chroma.shape))
    
        
# Também consegui reconstruir sinal a partir das mfccs, mas o código não está a funcionar de momento.

In [9]:
import plotly
import plotly.offline as py
plotly.offline.init_notebook_mode(connected=True)

import plotly.graph_objs as go
import plotly.figure_factory as ff
import colorlover as cl


In [10]:
pd.options.display.max_columns = 50
theta_glda_mirtools = pd.read_csv("/afs/l2f/home/frsc/Documents/ProjectosINESC/Fado/Full Report/Resources/phase1/Results/theta_gLDA_MFCC_mirtools.csv" ,sep=" ", header=None)
theta_glda_mirtools = theta_glda_mirtools.T


In [7]:
def tten(x):
    return int(x*9)

#theta samples x n_topics
def plotTheta(theta, num_songs):
    colors = cl.scales['9']['seq']['Reds']
    t = theta
    t['index_col'] = t.index
    t = t[t.columns.tolist()[-1:] + t.columns.tolist()[:-1]]
    col_n = len(t.columns)
    row_n = len(t.index)
    t_cols = [t[r].apply(tten)[:num_songs] for r in t.columns]
    t_cols_color = t_cols
    t_cols_color[0] = t_cols_color[1]
    

    trace0 = go.Table(
      header = dict(
        values = list(range(col_n)),
        line = dict(color = 'white'),
        fill = dict(color = 'white'),
        align = 'center',
        font = dict(color = 'black', size = 12)
      ),
      cells = dict(
        values = t_cols,
        line = dict(color = [np.array(colors)[np.array(c)] for c in t_cols_color]),
        fill = dict(color = [np.array(colors)[np.array(c)] for c in t_cols_color]),
        align = 'center',
        font = dict(color = 'white', size = 12)
        ))

    data = [trace0]

    py.iplot(data, filename = "cell variable color")

In [8]:
plotTheta(theta_glda_mirtools,theta_glda_mirtools.shape[1])

<b>Figura 3:</b>  Valores são arredondamentos a inteiros para lhes poder atribuir cores numa tabela do plotly.

In [9]:
theta_glda_mirtools.describe()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,index_col
count,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,741.000000,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,7.410000e+02,741.000000
mean,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,6.315357e-03,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,0.993555,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,3.131209e-06,1.355196e-05,3.131209e-06,370.000000
std,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,3.018236e-02,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,0.030208,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.148827e-06,1.648472e-04,1.148827e-06,214.052564
min,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,0.468923,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,8.933996e-07,0.000000
25%,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.534469e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,0.999839,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.466213e-06,2.473044e-06,2.466213e-06,185.000000
50%,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.152585e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,0.999877,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.001921e-06,3.006253e-06,3.001921e-06,370.000000
75%,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,4.124051e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,0.999901,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.647505e-06,3.664614e-06,3.647505e-06,555.000000
max,1.931994e-05,1.931994e-05,1.931994e-05,1.931994e-05,1.931994e-05,1.931994e-05,1.931994e-05,1.931994e-05,1.931994e-05,1.931994e-05,1.931994e-05,1.931994e-05

Como podemos observar, apenas os tópicos 15, 21, e 38 tomam valores com ordem de magnitude superior a -5 para alguma canção. Isto sugere mau funcionamento do gLDA já que a integridade dos dados foi assegurada pela sua geração de raiz.

Entretanto comecei a usar python, aprendi numpy, pandas, matplotlib, e sklearn para acelerar o fluxo de trabalho. O refazer das features não ajudou.

Sem solução em vista, escolhi discretizar as frames dos mfccs com kmeans para lhes aplicar um LDA discreto. Isto funcionou melhor, com resultados de aspecto natural, mas sem clusters óbvios. Antecipa-se que os resultados encontrados reflictam regularidades na qualidade de gravação mais do que aspectos musicais.


In [11]:
wavDir = "/ffs/tmp/frsc/fado/fado_init/wav/usable"
mfccDir = "/ffs/tmp/frsc/fado/fado_init/features3/mfcc/"
chromaDir = "/ffs/tmp/frsc/fado/fado_init/features2/chroma/"
labelDir = "/afs/l2f/home/frsc/Documents/ProjectosINESC/Fado/Full Report/Resources/phase1/labels.txt"

N = 700


#Directórios para teste com apenas 2 ficheiros de som
#wavDir = "/afs/l2f/home/frsc/Documents/ProjectosINESC/Fado/Full Report/Resources/isft/"
#mfccDir = "/afs/l2f/home/frsc/Documents/ProjectosINESC/Fado/Full Report/Resources/isft/"
#chromaDir = "/afs/l2f/home/frsc/Documents/ProjectosINESC/Fado/Full Report/Resources/isft/"
#labelDir = "/afs/l2f/home/frsc/Documents/ProjectosINESC/Fado/Full Report/Resources/isft/label_example.txt"

featureDir=mfccDir

In [12]:
#featureTypeName: nome do tipo de feature, lowercase
#idFromName: deduzir ID pelo nome qd tem formato "0001_featuretype.csv"
def buildSongsDF(featureDir, labelDir, featureTypeName, n_songs = 0, idFromName=False):
    # Load Labels
    labels_df = pd.read_csv(labelDir, delimiter = ' - ', header=0, index_col = 0, dtype=str)
    
    # Make filenames
    desiredSuffix = featureTypeName+".csv"
    files = [join(featureDir, f) for f in listdir(featureDir) if isfile(join(featureDir, f)) and f.endswith(desiredSuffix)]
    
    # Get IDs of songs to be loaded
    song_ids = [(os.path.basename(file)[:-len(desiredSuffix)-1]) for file in files]
    if n_songs == 0:
        song_n = len(song_ids)
    song_n = n_songs

    
    # load songs
    song_list = [None] * song_n
    for x in range(song_n):
        song_list[x] = pd.read_csv(files[x], header=None, index_col=None)
        
    song_ids = song_ids[:song_n]
    song_dict = dict(zip(song_ids, song_list))

    songs_df = pd.concat(song_dict)
    return song_ids, songs_df

In [13]:
song_ids, songs_df = buildSongsDF(mfccDir, labelDir, n_songs=N, featureTypeName="mfcc")


/afs/l2f.inesc-id.pt/home/frsc/virtualenv/lib/python3.4/site-packages/ipykernel_launcher.py:5: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



In [14]:
def discretizeContinuousFeatures(featureDir, labelDir, n_songs=0, cluster_n=100, featureTypeName="mfcc"):
    
    #KMeans
    song_ids, songs_df = buildSongsDF(featureDir, labelDir, n_songs = n_songs, featureTypeName = featureTypeName)
    songs_df_scaled = pd.DataFrame(preprocessing.scale(songs_df))
    song_n = len(song_ids)
    
    X_NEW = MiniBatchKMeans(n_clusters=cluster_n, random_state=0).fit_predict(songs_df_scaled)
    x_df = pd.DataFrame(X_NEW)
    x_df.groupby(0).size()

    # Reconstruir Canções com isto
    song_lens = [len(songs_df.loc[x]) for x in song_ids]
    
    discrete_songs = [None]*song_n
    high = 0
    low = 0
    for i in range(song_n):
        high += song_lens[i]
        discrete_songs[i] = X_NEW[low:high]
        low = high
        
    kmeansDir = featureDir + "meta/fadoKmeans.csv"
    
    x_df.to_csv(path_or_buf=kmeansDir, index=False)

    print("Document lengths:\n")
    for i in range(song_n):
        print("Song {0}: {1}".format(song_ids[i],len(discrete_songs[i])))
    return x_df, discrete_songs

In [15]:
x_df, discrete_songs = discretizeContinuousFeatures(mfccDir, labelDir, n_songs = N, featureTypeName="mfcc")

/afs/l2f.inesc-id.pt/home/frsc/virtualenv/lib/python3.4/site-packages/ipykernel_launcher.py:5: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



Document lengths:

Song 0001: 5430
Song 0054: 6794
Song 0055: 13523
Song 0057: 6766
Song 0058: 8043
Song 0059: 9057
Song 0060: 7627
Song 0061: 8810
Song 0062: 6993
Song 0063: 6787
Song 0064: 6498
Song 0065: 6215
Song 0066: 6857
Song 0067: 7623
Song 0068: 6206
Song 0069: 7363
Song 0070: 7663
Song 0071: 6379
Song 0073: 11147
Song 0075: 9885
Song 0076: 9142
Song 0077: 6673
Song 0078: 11707
Song 0079: 11051
Song 0080: 8835
Song 0081: 7400
Song 0082: 12859
Song 0083: 10048
Song 0084: 11111
Song 0085: 8312
Song 0086: 8544
Song 0087: 12857
Song 0088: 8021
Song 0089: 5781
Song 0090: 6599
Song 0091: 8237
Song 0094: 8268
Song 0095: 7197
Song 0096: 14078
Song 0097: 6978
Song 0098: 6281
Song 0099: 9863
Song 0100: 7812
Song 0102: 6169
Song 0104: 6905
Song 0105: 7499
Song 0106: 8535
Song 0107: 9233
Song 0109: 8177
Song 0110: 6327
Song 0111: 6739
Song 0112: 5518
Song 0113: 6359
Song 0114: 6743
Song 0115: 7470
Song 0116: 6552
Song 0117: 8410
Song 0118: 7959
Song 0119: 10699
Song 0121: 8748
Song 0122: 

In [16]:
#Frequências de cada "palavra"
x_df[0].value_counts()

10    150616
76    120843
78    118402
77    104212
66    102845
43    101225
11    100655
50     99935
15     97126
6      97013
20     95700
1      94919
30     92310
83     91344
46     91258
63     90534
12     90404
23     88942
86     87877
91     86767
80     83661
3      83368
32     82724
89     79667
42     78178
4      77132
25     76233
99     76025
59     73298
39     72993
       ...  
73     45517
90     45427
44     45333
62     44024
96     43875
68     42809
55     42206
69     41252
33     41119
61     40111
65     38984
19     38822
72     38810
98     38619
79     38455
8      38110
37     37944
57     37693
21     37682
71     36955
47     34841
5      34354
34     33314
67     30784
52     29630
75     25626
53     17425
35     14414
84     12323
87      4889
Name: 0, Length: 100, dtype: int64

In [17]:
# Fazer document-word matrix
from sklearn.feature_extraction import DictVectorizer
from collections import Counter, OrderedDict

def makeDocWordMatrix(mfccDir, discrete_songs):

    vectorizer = DictVectorizer()
    # discover corpus and vectorize file word frequencies in a single pass
    dwm = vectorizer.fit_transform(Counter(f) for f in (discrete_songs))
    #dwm = pd.DataFrame(dwm)
    #dwm.to_csv(mfccDir+"meta/DWM.csv")
    return dwm


In [18]:
dwm = makeDocWordMatrix(mfccDir, discrete_songs)


In [19]:
from sklearn.decomposition import LatentDirichletAllocation

def runLDA(dwm, outputPath):
    lda = LatentDirichletAllocation(n_components=40, random_state=0, evaluate_every=10)
    theta = lda.fit_transform(dwm)

    theta = pd.DataFrame(theta)
    theta.describe()
    theta.to_csv(outputPath)
    return theta
    

In [20]:
theta = runLDA(dwm, mfccDir+"meta/LDAtheta.csv")
theta.to_csv("test_theta.csv")

/afs/l2f.inesc-id.pt/home/frsc/virtualenv/lib/python3.4/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning:

The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.



In [21]:
theta.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
count,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000
mean,0.005826,0.031674,0.046758,0.025292,0.011477,0.033794,0.023719,0.001447,0.017634,0.050868,0.022814,0.038605,0.029423,0.022710,0.026640,0.031428,0.035491,0.011085,0.024344,0.018763,0.025038,0.064233,0.015235,0.048078,0.008161,0.033106,0.045744,0.038967,0.034164,0.017857,0.028080,0.003353,0.020163,0.012385,0.012566,0.014991,0.029663,0.000143,0.007305,0.030973
std,0.052005,0.082436,0.096064,0.046719,0.066158,0.079846,0.048168,0.037717,0.076560,0.094587,0.065732,0.089956,0.065163,0.054161,0.080004,0.066181,0.078865,0.039979,0.066149,0.070241,0.066319,0.148552,0.053927,0.098019,0.070691,0.078049,0.095987,0.088381,0.103449,0.067087,0.061224,0.054060,0.057781,0.035456,0.051501,0.085248,0.067946,0.000597,0.073541,0.062882
min,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001
25%,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000002,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000002,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003
50%,0.000004,0.000004,0.000004,0.008866,0.000003,0.000005,0.000004,0.000003,0.000004,0.000004,0.000004,0.000004,0.000005,0.000004,0.000004,0.000004,0.000004,0.000003,0.000005,0.000004,0.000004,0.000004,0.000004,0.000004,0.000003,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000003,0.000004,0.000004,0.000003,0.000004,0.000004,0.000003,0.000003,0.000004
75%,0.001198,0.016729,0.038303,0.029858,0.000005,0.024754,0.024151,0.000004,0.001855,0.061073,0.018900,0.023924,0.027966,0.015020,0.009545,0.032436,0.038693,0.000432,0.018922,0.006979,0.018050,0.040295,0.010936,0.046384,0.000007,0.028921,0.042580,0.022701,0.009824,0.007680,0.025650,0.000004,0.009629,0.007281,0.000004,0.002170,0.022088,0.000004,0.000184,0.033976
max,0.999926,0.715888,0.616553,0.385562,0.981574,0.705563,0.362549,0.997908,0.948254,0.646496,0.952000,0.736971,0.563070,0.421773,0.778845,0.503282,0.723802,0.618247,0.896111,0.814119,0.753852,0.923873,0.852975,0.578474,0.999884,0.726082,0.802066,0.558950,0.835848,0.873393,0.474867,0.999847,0.650423,0.478418,0.502057,0.988143,0.556531,0.006233,0.998313,0.475093


Esta matriz theta já parece mais natural! Procedamos à visualização

In [22]:
theta = pd.read_csv("test_theta.csv", header=0, index_col=0)

In [24]:
# Visualization

from os import listdir
from os.path import isfile, join
import numpy as np; np.random.seed(1)
from sklearn.manifold import TSNE


def decodeLabels(labelDir, labelMask=[None]):
    with open(labelDir, 'r') as labels:
        lines = labels.readlines()[1:]
        i = 0
        if all(x == None for x in labelMask):
            song_n = len(lines)
            labelMask = [x + 1 for x in range(song_n)]
        else:
            song_n = len(labelMask)
        songNames = [None] * song_n
        artistNames = [None] * song_n
        ids = [None] * song_n
        for a in labelMask:
            x = a - 1
            line = lines[x]
            s = line.split('-')
            ids[i] = a
            songNames[i] = s[2]
            artistNames[i] = s[1]
            i += 1
    return ids, songNames, artistNames



In [25]:
# Directório com labels id - artista - música
labelDir = "/afs/l2f/home/frsc/Documents/ProjectosINESC/Fado/Full Report/Resources/phase1/labels.txt"
labelMask = [int(x) for x in song_ids]


In [26]:
files = [join(featureDir, f) for f in listdir(featureDir) if isfile(join(featureDir, f))]
n_samples = len(files)

In [27]:

X_embedded = TSNE(n_components=2, perplexity=2.0).fit_transform(theta)
print(X_embedded.shape)

X_embedded2 = TSNE(n_components=2, perplexity=15.0).fit_transform(theta)
print(X_embedded.shape)

X_embedded3 = TSNE(n_components=2, perplexity=50.0).fit_transform(theta)
print(X_embedded.shape)

X_embedded4 = TSNE(n_components=2, perplexity=100.0).fit_transform(theta)
print(X_embedded.shape)

(700, 2)
(700, 2)
(700, 2)
(700, 2)


In [28]:
# Top Thetas
topTheta = theta.idxmax(1)
c = topTheta
ids, songNames, artistNames = decodeLabels(labelDir, labelMask)

In [29]:
x1 = X_embedded[:,0]
y1 = X_embedded[:,1]

x2 = X_embedded2[:,0]
y2 = X_embedded2[:,1]

x3 = X_embedded3[:,0]
y3 = X_embedded3[:,1]

x4 = X_embedded4[:,0]
y4 = X_embedded4[:,1]

In [30]:
names = ["{} - [{}] {} - {}\n".format(c[i], ids[i], songNames[i][:-1], artistNames[i]) for i in range(N)]


In [31]:
name_dict = {
    'top Topic': c,
    'id': ids,
    'song': songNames,
    'artist': artistNames
}
ndf = pd.DataFrame(name_dict)
ndf

,artist,id,song,top Topic
0,Amália Rodrigues,1,O Malmequer pequenino\n,26
1,Amália Rodrigues,54,Partindo Se (Lyrics)\n,2
2,Amália Rodrigues,55,fado portugal amalia\n,26
3,Amália Rodrigues,57,Malhão De Águeda\n,17
4,Amália Rodrigues,58,Acho Inúteis as Palavras (Lyrics _ texto)\n,2
5,Amália Rodrigues,59,Os teus olhos são dois círios\n,18
6,Amália Rodrigues,60,Madrugada de Alfama\n,2
7,Amália Rodrigues,61,'ALFAMA'\n,3
8,Amália Rodrigues,62,Maria Lisboa\n,23
9,Amália Rodrigues,63,uma casa portuguesa\n,23


In [32]:
ndf[:4]

,artist,id,song,top Topic
0,Amália Rodrigues,1,O Malmequer pequenino\n,26
1,Amália Rodrigues,54,Partindo Se (Lyrics)\n,2
2,Amália Rodrigues,55,fado portugal amalia\n,26
3,Amália Rodrigues,57,Malhão De Águeda\n,17


In [33]:
import plotly.graph_objs as go

def plotTSNE(x,y,c,N):  
    data = [
        go.Scatter(
            x = x,
            y = y,
            text = names,
            mode='markers',
            hoverinfo = 'text',
            marker = dict(
                color = [int(x)/(N/5) for x in c],
                colorscale='Viridis'
            ),
            showlegend = False
        )
    ]
    py.iplot(data, filename = "add-hover-text")

In [34]:
plotTSNE(x1,y1,c,N)
plotTSNE(x2,y2,c,N)
plotTSNE(x3,y3,c,N)
plotTSNE(x4,y4,c,N)

Figura 4. tSNE bidimensional para as misturas de tópicos com perplexities de 2, 15, 50, 100

In [35]:
plotTSNE(x2,y2,c,N)


Figura 5. O tsne com clusters mais diferenciados parece ser o de perplexidade 15

In [36]:
theta.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
count,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000
mean,0.005826,0.031674,0.046758,0.025292,0.011477,0.033794,0.023719,0.001447,0.017634,0.050868,0.022814,0.038605,0.029423,0.022710,0.026640,0.031428,0.035491,0.011085,0.024344,0.018763,0.025038,0.064233,0.015235,0.048078,0.008161,0.033106,0.045744,0.038967,0.034164,0.017857,0.028080,0.003353,0.020163,0.012385,0.012566,0.014991,0.029663,0.000143,0.007305,0.030973
std,0.052005,0.082436,0.096064,0.046719,0.066158,0.079846,0.048168,0.037717,0.076560,0.094587,0.065732,0.089956,0.065163,0.054161,0.080004,0.066181,0.078865,0.039979,0.066149,0.070241,0.066319,0.148552,0.053927,0.098019,0.070691,0.078049,0.095987,0.088381,0.103449,0.067087,0.061224,0.054060,0.057781,0.035456,0.051501,0.085248,0.067946,0.000597,0.073541,0.062882
min,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001
25%,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000002,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000002,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003
50%,0.000004,0.000004,0.000004,0.008866,0.000003,0.000005,0.000004,0.000003,0.000004,0.000004,0.000004,0.000004,0.000005,0.000004,0.000004,0.000004,0.000004,0.000003,0.000005,0.000004,0.000004,0.000004,0.000004,0.000004,0.000003,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000003,0.000004,0.000004,0.000003,0.000004,0.000004,0.000003,0.000003,0.000004
75%,0.001198,0.016729,0.038303,0.029858,0.000005,0.024754,0.024151,0.000004,0.001855,0.061073,0.018900,0.023924,0.027966,0.015020,0.009545,0.032436,0.038693,0.000432,0.018922,0.006979,0.018050,0.040295,0.010936,0.046384,0.000007,0.028921,0.042580,0.022701,0.009824,0.007680,0.025650,0.000004,0.009629,0.007281,0.000004,0.002170,0.022088,0.000004,0.000184,0.033976
max,0.999926,0.715888,0.616553,0.385562,0.981574,0.705563,0.362549,0.997908,0.948254,0.646496,0.952000,0.736971,0.563070,0.421773,0.778845,0.503282,0.723802,0.618247,0.896111,0.814119,0.753852,0.923873,0.852975,0.578474,0.999884,0.726082,0.802066,0.558950,0.835848,0.873393,0.474867,0.999847,0.650423,0.478418,0.502057,0.988143,0.556531,0.006233,0.998313,0.475093


In [37]:
import copy

class TopicModel:
    def __init__(self, theta, label_df):
        
        self.theta = theta
        self.label_df = label_df
        self.topTopics = theta.idxmax(1)
    
    def topicHead(self, topic_n, n_songs = 5):
        #Sort values of given topic
        topics_df = self.theta[str(topic_n)]
        topics_df = topics_df.sort_values(ascending=False)
        idx = topics_df.index
        #Labels for selected 
        return_df = copy.deepcopy(self.label_df)
        return_df["Topic Val"] = topics_df       
        return return_df.loc[idx[0:n_songs]]
    
    def largestTopics(self, i = 5):
        t = self.theta
        return t.sum().sort_values(ascending=False)[:i]
    
    # i: no. topics, j: no. songs per topic
    def exploreLargestTopics(self, n_topics = 5, n_songs = 5):
        l_topics = model.largestTopics(40)
        head_list = [model.topicHead(n,n_songs) for n in l_topics.index[:n_topics]]
        for x in l_topics.index[:n_topics]:
            print("\nTopic: " + str(x) +" : " + str(l_topics.loc[x]) + " total topic weight - means nothing")
            display(model.topicHead(x,n_songs))
        return head_list
    
    def topicComposition(self, song_id, n_topics = 5):
        #find theta line from id
        id_theta = copy.deepcopy(self.theta)
        id_theta["id"] = self.label_df["id"]
        return id_theta[id_theta["id"]==song_id]

In [38]:
model = TopicModel(theta, ndf)

In [39]:
display(model.topicHead(1))

,artist,id,song,top Topic,Topic Val
484,Vicente da Câmara,835,FADO DAS CALDAS\n,1,0.715888
557,José Pracana,910,Porque choraste por mim\n,1,0.620209
558,José Pracana,911,Primavera perdida\n,1,0.538644
583,Francisco Stoffel,937,Por Morrer uma Amdorinha\n,1,0.527115
556,José Pracana,909,Nocturno\n,1,0.521345


In [40]:
l_topics = model.largestTopics(40)
l_topics

21    44.963274
9     35.607525
23    33.654281
2     32.730748
26    32.020995
27    27.276864
11    27.023813
16    24.843904
28    23.914870
5     23.655728
25    23.174322
1     22.171778
15    21.999737
39    21.681126
36    20.764104
12    20.596001
30    19.656143
14    18.647994
3     17.704525
20    17.526689
18    17.040924
6     16.603556
10    15.969547
13    15.896846
32    14.113935
19    13.133918
29    12.499853
8     12.343911
22    10.664283
35    10.493598
34     8.796494
33     8.669449
4      8.034114
17     7.759748
24     5.712661
38     5.113365
0      4.078527
31     2.347197
7      1.013212
37     0.100440
dtype: float64

In [41]:
model.topicHead(11,6)

,artist,id,song,top Topic,Topic Val
636,João Ferreira Rosa,991,Fado do Sonho\n,11,0.736971
614,Gonçalo Salgueiro,968,Grito\n,11,0.539436
318,António Zambujo,624,Não é tarde nem é cedo\n,11,0.520150
343,Joana Mendes,649,Fado Magala 'Porque teimas nesta dor'\n,11,0.513621
301,Grupo de Fados do ISEP,606,"Adeus, adeus\n",11,0.453193
245,Pedro Viola,544,"Maria la Portuguesa, 'Fado TV' (2011)\n",11,0.444673


In [42]:
largest_topics = model.exploreLargestTopics(40,15)


Topic: 21 : 44.963273515424355 total topic weight - means nothing


,artist,id,song,top Topic,Topic Val
154,Mariza,229,Oiça lá ó Senhor Vinho\n,21,0.923873
482,Rodrigo,833,'Recordações do Passado'\n,21,0.922806
322,Teresa Lopes Alves,628,Carta do Fundo do Mar\n,21,0.867858
401,Ada de Castro,718,Gosto de tudo o que é teu\n,21,0.833671
185,Ana Moura,262,Dia De Folga\n,21,0.811281
572,Manuela Cavaco,925,Corrido antigo\n,21,0.803928
133,Mariza,205,Beijo de Saudade\n,21,0.784572
164,Raquel Tavares,239,Meu Amor de Longe\n,21,0.723180
175,Ana Moura,250,'DESFADO'\n,21,0.703418
349,Dulce Pontes,657,Fado Português\n,21,0.652918



Topic: 9 : 35.607524969659536 total topic weight - means nothing


,artist,id,song,top Topic,Topic Val
501,António Mourão,852,Estranha Contradicao\n,9,0.646496
517,Fernando Farinha,868,CANTANDO UM BELO FADO NO FILME 'A ULTIMA PEGA'\n,9,0.497774
475,Fernando Maurício,826,Fado na Mouraria\n,9,0.487167
505,António Mourão,856,'A Noite'\n,9,0.441785
99,Amália Rodrigues,165,Oiça lá ó senhor vinho (with lyrics_ com text...,9,0.437631
504,António Mourão,855,Eu canto enquando soubér cantar\n,9,0.437378
341,Maria Fernanda Soares,647,MARIA FERNANDA SOARES E ARTUR GARCIA\n,9,0.437363
388,Carlos do Carmo,698,Lisboa menina e moça\n,9,0.413399
564,Maria Armanda,917,A Bia da Mouraria\n,9,0.407457
671,Berta Cardoso,1026,'Noite de S. João'\n,9,0.391134



Topic: 23 : 33.654281109074205 total topic weight - means nothing


...

,artist,id,song,top Topic,Topic Val
17,Amália Rodrigues,71,Oiça Lá Ó Senhor Vinho\n,31,0.999847
49,Amália Rodrigues,110,Oiça lá ò Sr. vinho\n,31,0.999846
496,Lucília do Carmo,847,Foi na travessa da palha\n,8,0.216196
432,Ada de Castro e Geraldo da Viola & Osvaldo,775,Fado_O Grito Que o Vento Leva\n,19,0.062835
538,Carlos Macedo,891,'Avé Maria Fadista'\n,25,0.015889
134,Mariza,206,Há palavras que nos beijam\n,27,0.012106
423,Ada de Castro,760,Na crista daquela onda\n,13,0.009512
153,Mariza,227,Boa Noite solidão Mariza\n,27,0.007094
301,Grupo de Fados do ISEP,606,"Adeus, adeus\n",11,0.004549
687,Fábia Rebordão,1042,Por Sombras me Dei à Luz\n,21,0.003855



Topic: 7 : 1.0132122958005703 total topic weight - means nothing


,artist,id,song,top Topic,Topic Val
533,Luiz Goes,885,Toada Beirã\n,7,0.997908
247,Nathalie Pires,547,Estranha Forma de Vida\n,26,0.005274
141,Mariza,213,Por ti (Fado tango)\n,10,0.003130
109,Mariza,176,'Os anéis do meu cabelo'\n,29,0.002943
250,Ana Margarida,550,Carmencita\n,23,0.001013
404,Ada de Castro,724,O melhor bem (Arlindo de Carvalho)\n,30,0.000342
136,Mariza,208,"povo que lavas no rio, mariza, fado\n",23,0.000318
438,Ada de Castro,784,Depoimento 01\n,18,0.000114
420,Ada de Castro,754,Karaoke Fado GOSTO DE TUDO O QUE É TEU\n,27,0.000019
447,Hermínia Silva,796,FADO ROSA MARIA\n,2,0.000008



Topic: 37 : 0.10043960907302604 total topic weight - means nothing


,artist,id,song,top Topic,Topic Val
354,Katia Guerreiro,663,Guitarra Triste\n,13,0.006233
377,Maria da Fé,686,Até que a voz me doa\n,36,0.005980
289,autor desconhecido coimbra,592,Coimbra Samaritana\n,39,0.004781
139,Mariza,211,Maria Lisboa (with pictures of Lisbon)\n,23,0.004495
108,Mariza,175,Alguém me Ouviu (Mantém te Firme)\n,21,0.004471
574,Manuela Cavaco,927,Brincos para brincar\n,21,0.003701
573,Manuela Cavaco,926,Fado Velhinho\n,13,0.003556
397,Ada de Castro,713,Eu sou Filha de Lisboa\n,23,0.003311
215,MARGARIDA GUERREIRO,513,Libertação\n,16,0.003265
462,Frei Hermano da Câmara,813,"'Minha mãe, nasci fadista'\n",13,0.003186


Numa análise prematura podemos identificar:

o "topo" do tópico 5 como maioritariamente ocupado por obras de vozes femininas, 

o tópico 26 por vozes masculinas, 

o tópico 29 quase exclusivamente por Mariza e Ana Moura, duas cantoras contemporâneas (que sugere um tópico de fado mais recente ou pelo menos técnicas de produção modernas)

o tópico 25 sugere, pela inspecção da primeira música e das várias partes da serenata em Tokyo, seleccionar o ruído de uma gravação ao vivo de má qualidada

o tópico 33 agrupou 3 músicas de João Braga (de fontes diferentes) 




O modelo de tópicos resultante revela estar a encontrar certas regularidades, mas (à primeira vista) parece insuficiente para fazer algum tipo de modelação de emoções. Alguns tópicos concentram um ou dois artístas como de maior relevância para um tópico. Isto pode dever-se às propriedades de gravação das faixas de música, bem como à qualidade da gravação.

Uma das abordagens mais usadas para interpretar os resultados de um LDA é olhando para as palavras que carregam mais influência de cada tópico. Como estamos a lidar com "palavras" sonoras que duram uma fracção de segundo, não ganhamos muito com ouvi-las individualmente. (Poderíamos, no entanto, tentar ouvir regiões grandes das faixas que estejam mapeadas para o mesmo tópico) 

Uma análise mais profunda envolveria o escutar atento de músicas. Obter metadados em relação à região e estilo do fado ajudaria a obter conclusões  interessantes.